In [6]:
#Libaries to Import
import os   
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import SARIMAX
from math import sqrt

In [98]:
#Data 
all_files = sorted(os.listdir("data"))
pnl_files = [f for f in all_files if f.startswith("PnL")]
df = pd.concat((pd.read_csv(os.path.join("data", f), delimiter=";", encoding="ISO-8859-1", decimal = ",", thousands='.') for f in pnl_files))

#Renmane columns
df.rename (columns= {'Calendar_Year': 'Year'}, inplace=True)
df.rename (columns= {'Calendar_Month': 'Month'}, inplace=True)

# Create a year_month variable as a period which respects calendar time ordering
df['Year_Month'] = pd.to_datetime(df['Year'].astype(str)  + df['Month'], format='%Y%B')

print(df.head(6))


   Year    Month          Account_Text  Account_Key   Amount Year_Month
0  2015  January  COGS (excl. Mfg. Ove         2000  -515.31 2015-01-01
1  2015  January     Distribution Cost         3000   -88.78 2015-01-01
2  2015  January         Gross Revenue         1100  3135.52 2015-01-01
3  2015  January       Marketing Costs         4000  -107.23 2015-01-01
4  2015  January    Operating Expenses         5000  -804.72 2015-01-01
5  2015  January       Sales Discounts         1400   -79.07 2015-01-01
